In [1]:
import os
import json
import torch, json

base_path = '/media/melissa/EXTERNAL_USB/docred_output' 

pred_path = os.path.join(base_path, "predictions.jsonl")
test_path = os.path.join(base_path, "test.pt")

In [2]:
def load_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [3]:
test = torch.load(test_path)

/tmp/ipykernel_16031/343780022.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test = torch.load(test_path)


In [4]:
predictions = []
with open(pred_path, "r") as f:
    for line in f:
        predictions.append(json.loads(line))


In [5]:
# Ground truth
title_to_gold = {
    item["title"]: set(
        tuple(label) for label in item.get("labels", [])
        if label[2] != 0  # filter out NA
    )
    for item in test
}


title_to_pred = {
    item["title"]: set(
        tuple(label) for label in item.get("labels", [])
        if label[2] != 0  # filter out NA
    )
    for item in predictions
}

In [6]:
total_preds = sum(len(v) for v in title_to_pred.values())
print("Total predicted relations (excluding NA):", total_preds)
total_golds = sum(len(v) for v in title_to_gold.values())
print("Total gold (ground-truth) relations:", total_golds)


Total predicted relations (excluding NA): 3384
Total gold (ground-truth) relations: 85932


In [7]:
len(title_to_pred)

3053

In [8]:
len(title_to_gold)

3053

In [9]:
tp = 0
fp = 0
fn = 0

for title in title_to_gold:
    gold = title_to_gold.get(title, set())
    pred = title_to_pred.get(title, set())
    
    tp += len(gold & pred)
    fp += len(pred - gold)
    fn += len(gold - pred)

precision = tp / (tp + fp + 1e-10)
recall = tp / (tp + fn + 1e-10)
f1 = 2 * precision * recall / (precision + recall + 1e-10)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Precision: 0.0050
Recall:    0.0002
F1 Score:  0.0004


In [10]:
total_gold = sum(len(v) for v in title_to_gold.values())
total_pred = sum(len(v) for v in title_to_pred.values())
overlap = sum(len(title_to_gold[title] & title_to_pred[title]) for title in title_to_gold)

print(f"Total gold: {total_gold}")
print(f"Total predicted: {total_pred}")
print(f"Overlapping (TP): {overlap}")


Total gold: 85932
Total predicted: 3384
Overlapping (TP): 17


NameError: name 'train' is not defined